# Construcción RAG con Langchain y muestra de interfaz con Gradio

In [1]:
!pip install langchain
!pip install unstructured
!pip install chromadb
!pip install tiktoken
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.4/189.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 8.9 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=848ceb885c78f16bb1ba0b705ab0294f7d6d4ae4a103809564a63aaa718dd443
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf

In [2]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.52
    Uninstalling langchain-core-0.3.52:
      Successfully uninstalled langchain-core-0.3.52
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.23
    Uninstalling langchain-0.3.23:
      Successfully uninstalled langchain-0.3.23


In [3]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

import gradio as gr

In [4]:

# La descarga de la wiki de Sonneil la tenemos disponible en Drive
folder_path = "/content/drive/MyDrive/sonneil-wiki-master/docs"

# Load documents
loader = DirectoryLoader(folder_path, glob="*.md", recursive=True)
documents = loader.load()



In [5]:
# obtenemos el número de páginas o documentos contenidos en pages
len(documents)

171

Clave de API guardada en los Secrets del notebook

In [8]:
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [9]:
# inicializamos los embeddings de OpenAI
embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# Create a vectorstore
db = Chroma.from_documents(documents, embedding)

<ipython-input-9-9c73334cc7e6>:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)


In [10]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 2})

import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
llm = OpenAI(temperature=0)


<ipython-input-10-80250447be61>:5: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0)


In [11]:
# Create the RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)


In [12]:
# Example query
#query = "¿Cómo se crea un Sale Case?"
#result = qa_chain.run(query)
#result

In [13]:
# Example query
#query = "¿Qué me puedes decir sobre el Scoring?"
#result = qa_chain.run(query)
#result

In [14]:
# inicializamos el historial de chat para su incorporacion a
# visualización en gradio de toda la interacción
chat_history = []
# inicializamos el contexto para el modelo GPT de OpenAI
context = []


# creamos una función para obtener las respuestas del chatbot, que
# devuelve todo el historial del chat formateado para mostrarlo en Gradio
def chatbot_response(input_text):
  # definimos la variable chat_history como global
  global chat_history
  # definimos el mensaje del usuario al historial con un estilo de "burbuja" de chat
  user_message = f'<div class="talk-bubble tri-right left-in"><div class="talktext"><p><b>Usuario:</b> {input_text}</p></div></div>'
  # añadimos el mensaje del usuario a la historia del chat
  chat_history.append(user_message)
  # generamos la respuesta del modelo de lenguaje en base a la pregunta del usuario
  response_text = qa_chain.run(input_text)
  # Añadir la respuesta del bot al historial con un estilo de "burbuja" de chat
  bot_message = f'<div class="talk-bubble tri-right left-in"><div class="talktext"><p><b>Sonneil:</b> {response_text}</p></div></div>'
  # añadimos el mesaje del bot formateado al historial del chat
  chat_history.extend([bot_message])
  # formateamos el historial del chat para mostrarlo en la interfaz, poniendo
  # un scroll particular para visualizar un chat largo
  formatted_history = '<div style="overflow-y: scroll; height: 80vh;">' + "".join(chat_history) + '</div>'
  # devuelve la historia formateada
  return formatted_history


# definimos la interfaz de Gradio
iface = gr.Interface(
    fn=chatbot_response,
    inputs=gr.Textbox(lines=2, placeholder="Escribe tu consulta aquí...", label="Realiza una consulta al Chatbot:"),
    outputs=gr.HTML(),
    title='Sonneil Chatbot',
    description='Chatbot de consulta Sonneil',
    flagging_mode='never',
    css="""
    body{
  background-color: lightblue;
  font-family: "Ubuntu-Italic", "Lucida Sans", helvetica, sans;
}

/* container */
.container {
  padding: 5% 5%;
}

/* CSS talk bubble */
.talk-bubble {
	margin: 40px;
  display: inline-block;
  position: relative;
	width: 200px;
	height: auto;
	background-color: #cbe6e6;
}
.border{
  border: 8px solid #666;
}
.round{
  border-radius: 30px;
	-webkit-border-radius: 30px;
	-moz-border-radius: 30px;

}

/* Right triangle, left side slightly down */
.tri-right.border.left-in:before {
	content: ' ';
	position: absolute;
	width: 0;
	height: 0;
  left: -40px;
	right: auto;
  top: -8px;
	bottom: auto;
	border: 32px solid;
	border-color: #666 transparent transparent transparent;
}
.tri-right.left-in:after{
	content: ' ';
	position: absolute;
	width: 0;
	height: 0;
  left: -20px;
	right: auto;
  top: 0px;
	bottom: auto;
	border: 22px solid;
	border-color: #cbe6e6 transparent transparent transparent;
}

/* talk bubble contents */
.talktext{
  padding: 1em;
	text-align: left;
  line-height: 1.5em;
}
.talktext p{
  /* remove webkit p margins */
  -webkit-margin-before: 0em;
  -webkit-margin-after: 0em;
}
    """
)

# lanzamos la interfaz de Gradio
iface.launch(debug=False, share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://061dbf5bb0ef696a36.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
